In [ ]:
from can.interface import Bus
import can
from cantools.database import load_file
from broqer import Sink
from canbro import Node, update_e2e_autosar_profile2
import ipywidgets as widgets

In [ ]:
# load dbc file
db = load_file('poc/device_CAN.dbc')

In [ ]:
# create ECU node with virtual bus test
bus_e= Bus('test', interface='virtual')
ecu = Node(name="ECU",bus=bus_e,database=db )

In [ ]:
# create VCU node with virtual bus test, and connect to ECU via same name of bus -> test
bus_v= Bus('test', interface='virtual')
vcu= Node(name='CONTROL',bus=bus_v,database=db )

In [ ]:
print(vcu.DEM._signal_operation_mode._metadata.initial)

In [ ]:
print(vcu.DEM._signal_DEM_CRC._metadata.initial)

In [ ]:
def update_E2E( msg: can.Message):
    # Get the message from the database using the arbitration ID
    message = db.get_message_by_frame_id(msg.arbitration_id)

    if "DEM" == message.name:
        update_e2e_autosar_profile2(msg)
        vcu.DEM._signal_DEM_CRC.notify(msg.data[0])
        vcu.DEM._signal_DEM_SNC.notify(msg.data[1] & 0x0F)

In [ ]:
vcu.DEM.start_periodic(msg_callback=update_E2E)

In [ ]:
label_value = widgets.HTML(value="value=?")
def show_vcu_value(value, label: widgets.Label):
    label.value = 'counter for ECU node DEM: <br> value={}'.format(value)

show_print = ecu.DEM._signal_DEM_SNC.subscribe(Sink(show_vcu_value, label=label_value))


In [ ]:
list_op_mode = list(set([str(x) for x in list(reversed(vcu.DEM._signal_operation_mode._metadata.conversion.choices.values()))]))

In [ ]:
# Check if the value is in the list
if str(vcu.DEM._signal_operation_mode._state) in list_op_mode:
    value = str(vcu.DEM._signal_operation_mode._state)
else:
    value = list_op_mode[0]  # Set a default value

vcu_mode = widgets.ToggleButtons(
    options=list_op_mode,
    description=ecu.DEM._signal_operation_mode._metadata.name + ' selection for VCU node:',
    disabled=False,
    button_style='', 
    tooltips=list_op_mode,
    value=value,
)
# 'success', 'info', 'warning', 'danger' or ''

In [ ]:
def set_vcu_mode(value):
    #print('set value to {}'.format(value))
    vcu.DEM._signal_operation_mode.notify(value['new'])
    
vcu_mode.observe(set_vcu_mode, 'value')

In [ ]:
def val_vcu_value(value):
    global a_op_mode
    a_op_mode = value

show_print_var = ecu.DEM._signal_operation_mode.subscribe(Sink(val_vcu_value))

In [ ]:
display(vcu_mode)
display(label_value)

In [ ]:
vcu.DEM._signal_DEM_SNC._state

In [ ]:
vcu.DEM._signal_DEM_CRC._state